# VINDR

In [21]:
import pandas as pd
import numpy as np
import os

In [22]:
train_df = pd.read_csv('/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/train_vindr.csv')
val_df = pd.read_csv('/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/val_vindr.csv')
test_df = pd.read_csv('/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/test_vindr.csv')

train_embeddings = pd.read_csv("/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/train_embeddings.csv")
val_embeddings = pd.read_csv("/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/val_embeddings.csv")
test_embeddings = pd.read_csv("/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/test_embeddings.csv")


In [23]:
# rename column no finding to No Finding
train_df.rename(columns={'No finding': 'No Finding'}, inplace=True) 
val_df.rename(columns={'No finding': 'No Finding'}, inplace=True)
test_df.rename(columns={'No finding': 'No Finding'}, inplace=True)

train_df.head()

,image_id,rad_ID,No Finding,Bronchitis,Brocho-pneumonia,Other disease,Bronchiolitis,Situs inversus,Pneumonia,Pleuro-pneumonia,Diagphramatic hernia,Tuberculosis,Congenital emphysema,CPAM,Hyaline membrane disease,Mediastinal tumor,Lung tumor
0,21f822601908fcbf8502bf2ec9acafa3,R6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,b30001186b0cac5c73b55325ab83205c,R4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,48e4e2ee3c8b778c39749c549f717cec,R6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ac2bb817cac06f82d4011adfd8e9fd25,R2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,741e1e56fbf905d15b266d2a769c2db2,R1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# extract image_id from Path of embeddings to merge with diseases
train_embeddings['image_id'] = train_embeddings['Path'].apply(lambda x: x.split('/')[-1].split('.')[0])
val_embeddings['image_id'] = val_embeddings['Path'].apply(lambda x: x.split('/')[-1].split('.')[0])
test_embeddings['image_id'] = test_embeddings['Path'].apply(lambda x: x.split('/')[-1].split('.')[0])

# remove column Unnamed: 0
test_embeddings.drop(columns=['Unnamed: 0'], inplace=True)

# add / to Path
train_embeddings['Path'] = train_embeddings['Path'].apply(lambda x: '/MedImageInsights/data/' + x)
val_embeddings['Path'] = val_embeddings['Path'].apply(lambda x: '/MedImageInsights/data/' + x)
test_embeddings['Path'] = test_embeddings['Path'].apply(lambda x: '/MedImageInsights/data/' + x)

test_embeddings.head()


,Path,0,1,2,3,4,5,6,7,8,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,image_id
0,/MedImageInsights/data/vindr-pcxr/1.0.0/test/d...,-0.021907,-0.009163,0.011742,0.008527,-0.005080,0.005113,-0.036603,0.013077,0.006964,...,-0.041257,-0.008329,-0.033226,-0.000509,0.015108,-0.020855,-0.013732,-0.032404,0.039179,d7e71a052a753c3f2f3e317d60177bec
1,/MedImageInsights/data/vindr-pcxr/1.0.0/test/e...,-0.024549,0.005733,0.006984,-0.004194,-0.026601,0.026517,-0.025308,0.022511,0.022022,...,-0.035836,-0.005105,-0.041323,0.000947,0.001165,-0.021253,-0.017483,-0.011085,0.062195,e64d9421e78c82b5c104021a6c1b4e67
2,/MedImageInsights/data/vindr-pcxr/1.0.0/test/8...,-0.024070,0.036973,0.010903,-0.008274,-0.014543,0.003867,-0.039218,0.012558,0.015043,...,-0.014503,-0.005334,-0.042515,0.030343,-0.013595,-0.021082,-0.021357,-0.011207,0.055346,8a12a4352a866aaea4691dd27a99c7ae
3,/MedImageInsights/data/vindr-pcxr/1.0.0/test/3...,-0.042228,0.026734,0.008338,-0.011821,-0.009598,-0.020534,-0.019952,0.012481,0.010396,...,-0.059959,0.023548,-0.071086,0.031947,-0.019579,0.012865,-0.011924,-0.015732,0.035909,30d6a0843a9792168fdc456c17a9028c
4,/MedImageInsights/data/vindr-pcxr/1.0.0/test/f...,-0.013392,0.011721,0.021234,-0.000151,-0.020884,0.002880,-0.035558,0.019454,0.016745,...,-0.047692,-0.003190,-0.069404,-0.000160,-0.007703,-0.023449,-0.025272,-0.026673,0.048573,f22138be08a24e79cf07f6207341e5f5


In [25]:
# get differing columns
train_cols = train_df.columns
val_cols = val_df.columns
test_cols = test_df.columns

train_cols.difference(val_cols), train_cols.difference(test_cols), val_cols.difference(test_cols)

train_cols = train_embeddings.columns
val_cols = val_embeddings.columns
test_cols = test_embeddings.columns

train_cols.difference(val_cols), train_cols.difference(test_cols), val_cols.difference(test_cols)

(Index([], dtype='object'),
 Index([], dtype='object'),
 Index([], dtype='object'))

In [26]:
print(train_df.shape, val_df.shape, test_df.shape)
print(train_embeddings.shape, val_embeddings.shape, test_embeddings.shape)

# merge on image_id 
train_df = pd.merge(train_df, train_embeddings, on='image_id')
val_df = pd.merge(val_df, val_embeddings, on='image_id')
test_df = pd.merge(test_df, test_embeddings, on='image_id')

print(train_df.shape, val_df.shape, test_df.shape)

# make Path as first column 
train_df = train_df[['Path'] + [col for col in train_df.columns if col != 'Path']]
val_df = val_df[['Path'] + [col for col in val_df.columns if col != 'Path']]
test_df = test_df[['Path'] + [col for col in test_df.columns if col != 'Path']]

# drop image_id
train_df.drop(columns=['image_id'], inplace=True)
val_df.drop(columns=['image_id'], inplace=True)
test_df.drop(columns=['image_id'], inplace=True)

test_df.head()


(7341, 17) (387, 17) (1397, 17)
(7341, 1026) (387, 1026) (1397, 1026)
(7341, 1042) (387, 1042) (1397, 1042)


,Path,rad_ID,No Finding,Bronchitis,Brocho-pneumonia,Other disease,Bronchiolitis,Situs inversus,Pneumonia,Pleuro-pneumonia,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,/MedImageInsights/data/vindr-pcxr/1.0.0/test/d...,R3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.030038,-0.041257,-0.008329,-0.033226,-0.000509,0.015108,-0.020855,-0.013732,-0.032404,0.039179
1,/MedImageInsights/data/vindr-pcxr/1.0.0/test/e...,R3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.026000,-0.035836,-0.005105,-0.041323,0.000947,0.001165,-0.021253,-0.017483,-0.011085,0.062195
2,/MedImageInsights/data/vindr-pcxr/1.0.0/test/8...,R3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.014811,-0.014503,-0.005334,-0.042515,0.030343,-0.013595,-0.021082,-0.021357,-0.011207,0.055346
3,/MedImageInsights/data/vindr-pcxr/1.0.0/test/3...,R3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.020713,-0.059959,0.023548,-0.071086,0.031947,-0.019579,0.012865,-0.011924,-0.015732,0.035909
4,/MedImageInsights/data/vindr-pcxr/1.0.0/test/f...,R3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.019441,-0.047692,-0.003190,-0.069404,-0.000160,-0.007703,-0.023449,-0.025272,-0.026673,0.048573


In [15]:
# save to csv
train_df.to_csv('/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/train.csv', index=False)
val_df.to_csv('/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/val.csv', index=False)
test_df.to_csv('/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/test.csv', index=False)

In [12]:
import pandas as pd

train_df = pd.read_csv('/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/train.csv')
val_df = pd.read_csv('/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/val.csv')
test_df = pd.read_csv('/mnt/data2/datasets_lfay/MedImageInsights/data/vindr-pcxr/test.csv')

print(train_df.shape, val_df.shape, test_df.shape)

(7341, 1041) (387, 1041) (1397, 1041)


In [6]:
# remove /MedImageInsights/data/vindr-pcxr from Path
train_df['Path'] = train_df['Path'].apply(lambda x: x.split('/MedImageInsights/data/vindr-pcxr')[-1])
val_df['Path'] = val_df['Path'].apply(lambda x: x.split('/MedImageInsights/data/vindr-pcxr')[-1])
test_df['Path'] = test_df['Path'].apply(lambda x: x.split('/MedImageInsights/data/vindr-pcxr')[-1])
train_df.head()

,Path,rad_ID,No Finding,Bronchitis,Brocho-pneumonia,Other disease,Bronchiolitis,Situs inversus,Pneumonia,Pleuro-pneumonia,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,/1.0.0/train/21f822601908fcbf8502bf2ec9acafa3.jpg,R6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.024687,-0.030444,0.003069,-0.055603,-0.003481,0.004572,-0.014521,-0.029270,-0.010988,0.063009
1,/1.0.0/train/b30001186b0cac5c73b55325ab83205c.jpg,R4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.027162,-0.041601,-0.000895,-0.041030,-0.003012,0.000055,-0.023444,-0.022683,-0.012469,0.065280
2,/1.0.0/train/48e4e2ee3c8b778c39749c549f717cec.jpg,R6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.031814,-0.035963,-0.010607,-0.047651,0.009997,0.009302,-0.014764,-0.008199,-0.022131,0.054052
3,/1.0.0/train/ac2bb817cac06f82d4011adfd8e9fd25.jpg,R2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.007634,-0.040367,0.004174,-0.046368,0.004639,-0.004564,-0.019701,-0.035018,-0.017919,0.050189
4,/1.0.0/train/741e1e56fbf905d15b266d2a769c2db2.jpg,R1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.014459,-0.038117,-0.009935,-0.033321,0.013490,-0.002956,-0.031113,-0.016274,-0.034960,0.039006


In [13]:
# add /vindr-pcxr to Path
train_df['Path'] = train_df['Path'].apply(lambda x: '/vindr-pcxr' + x)
val_df['Path'] = val_df['Path'].apply(lambda x: '/vindr-pcxr' + x)
test_df['Path'] = test_df['Path'].apply(lambda x: '/vindr-pcxr' + x)


In [14]:
train_df

,Path,rad_ID,No Finding,Bronchitis,Brocho-pneumonia,Other disease,Bronchiolitis,Situs inversus,Pneumonia,Pleuro-pneumonia,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,/vindr-pcxr/1.0.0/train/21f822601908fcbf8502bf...,R6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.024687,-0.030444,0.003069,-0.055603,-0.003481,0.004572,-0.014521,-0.029270,-0.010988,0.063009
1,/vindr-pcxr/1.0.0/train/b30001186b0cac5c73b553...,R4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.027162,-0.041601,-0.000895,-0.041030,-0.003012,0.000055,-0.023444,-0.022683,-0.012469,0.065280
2,/vindr-pcxr/1.0.0/train/48e4e2ee3c8b778c39749c...,R6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.031814,-0.035963,-0.010607,-0.047651,0.009997,0.009302,-0.014764,-0.008199,-0.022131,0.054052
3,/vindr-pcxr/1.0.0/train/ac2bb817cac06f82d4011a...,R2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.007634,-0.040367,0.004174,-0.046368,0.004639,-0.004564,-0.019701,-0.035018,-0.017919,0.050189
4,/vindr-pcxr/1.0.0/train/741e1e56fbf905d15b266d...,R1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.014459,-0.038117,-0.009935,-0.033321,0.013490,-0.002956,-0.031113,-0.016274,-0.034960,0.039006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7336,/vindr-pcxr/1.0.0/train/7d213b491fceeb02ca272b...,R4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.021450,-0.023695,-0.026113,-0.058597,0.027368,-0.004179,-0.013916,-0.026958,-0.006002,0.074117
7337,/vindr-pcxr/1.0.0/train/c360e96915fb8b180ae072...,R6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.027761,-0.023388,-0.005845,-0.043216,0.018814,-0.008668,-0.031421,-0.017828,0.002581,0.044911
7338,/vindr-pcxr/1.0.0/train/7d92ee650f4f9d11524809...,R3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.011232,-0.037526,-0.002146,-0.039572,0.006642,0.004156,-0.011603,-0.011296,-0.021053,0.050128
7339,/vindr-pcxr/1.0.0/train/4955f60f2c996a7b2fa3cb...,R5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.043060,-0.027500,-0.013324,-0.037240,0.031521,-0.016043,0.019190,-0.017965,-0.000011,0.021069
